

# 🚀 Auto-Tuner: Fine-Tune Llama 3 with Unsloth

This notebook fine-tunes Llama 3 8B on your generated dataset using Unsloth (2x faster, 60% less memory).

**Requirements:**
- Free T4 GPU (Runtime → Change runtime type → T4 GPU)
- Dataset uploaded to Google Drive

**Time:** ~20-30 minutes for 100 examples

## 📋 Configuration

**IMPORTANT:** Update these values before running!

In [1]:
# ============================================================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================================================

# Dataset filename (must exist in Drive: Finetune_Jobs/datasets/)
DATASET_FILENAME = "dataset-20251125_095111.jsonl"  # ← CHANGE THIS

# Model name (will be saved to Drive: Finetune_Jobs/models/)
MODEL_NAME = "financial gpt"  # ← CHANGE THIS

# Training settings
MAX_SEQ_LENGTH = 2048        # Context window size
BATCH_SIZE = 2               # Larger = faster but more memory
GRADIENT_ACCUMULATION = 4    # Effective batch size = 2 * 4 = 8
LEARNING_RATE = 2e-4         # Learning rate
NUM_EPOCHS = 3               # Training epochs
WARMUP_STEPS = 5             # Warmup steps

print("✅ Configuration loaded")
print(f"Dataset: {DATASET_FILENAME}")
print(f"Model: {MODEL_NAME}")

✅ Configuration loaded
Dataset: dataset-20251125_095111.jsonl
Model: financial gpt


## 🔗 Step 1: Mount Google Drive

In [2]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Set paths
DRIVE_ROOT = "/content/drive/MyDrive/Finetune_Jobs"
DATASET_PATH = f"{DRIVE_ROOT}/datasets/{DATASET_FILENAME}"
MODEL_OUTPUT_DIR = f"{DRIVE_ROOT}/models/{MODEL_NAME}"

# Create directories if they don't exist
os.makedirs(f"{DRIVE_ROOT}/datasets", exist_ok=True)
os.makedirs(f"{DRIVE_ROOT}/models", exist_ok=True)

# Verify dataset exists
if not os.path.exists(DATASET_PATH):
    raise FileNotFoundError(f"❌ Dataset not found: {DATASET_PATH}\n\nPlease upload {DATASET_FILENAME} to Drive: Finetune_Jobs/datasets/")

print(f"✅ Drive mounted")
print(f"✅ Dataset found: {DATASET_PATH}")
print(f"✅ Model will be saved to: {MODEL_OUTPUT_DIR}")

Mounted at /content/drive
✅ Drive mounted
✅ Dataset found: /content/drive/MyDrive/Finetune_Jobs/datasets/dataset-20251125_095111.jsonl
✅ Model will be saved to: /content/drive/MyDrive/Finetune_Jobs/models/financial gpt


## 📦 Step 2: Install Unsloth

In [3]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" "peft" "accelerate" "bitsandbytes"

print("✅ Unsloth installed")

## 🤖 Step 3: Load Base Model (Llama 3 8B)

In [4]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

# Fix: Set the chat template for Llama-3
tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|start_header_id|>user<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>' }}{% elif message['role'] == 'assistant' %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>' }}{% endif %}{% endfor %}"

print("✅ Base model loaded (Llama 3 8B 4-bit)")
print(f"Model size: ~4.5GB")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

✅ Base model loaded (Llama 3 8B 4-bit)
Model size: ~4.5GB
Max sequence length: 2048


## 🎛️ Step 4: Add LoRA Adapters

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("✅ LoRA adapters added")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Unsloth 2025.11.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ LoRA adapters added
Trainable parameters: 41,943,040


## 📊 Step 5: Load and Preview Dataset

In [6]:
from datasets import load_dataset
import json

dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

print(f"✅ Dataset loaded: {len(dataset)} examples")
print(f"\nFirst example:")
print(json.dumps(dataset[0], indent=2))

print(f"\n📋 Preview of conversations:")
for i in range(min(3, len(dataset))):
    messages = dataset[i]['messages']
    user_msg = next((m['content'] for m in messages if m['role'] == 'user'), '')
    assistant_msg = next((m['content'] for m in messages if m['role'] == 'assistant'), '')
    print(f"\nExample {i+1}:")
    print(f"  User: {user_msg[:80]}...")
    print(f"  Assistant: {assistant_msg[:80]}...")

Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset loaded: 100 examples

First example:
{
  "messages": [
    {
      "role": "user",
      "content": "Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3 years, 12% discount rate, 2% terminal growth"
    },
    {
      "role": "assistant",
      "content": "DCF Valuation Analysis:\nProjected Cash Flows:\nYear 1: $18.75M (25% growth)\nYear 2: $23.44M (25% growth)\nYear 3: $29.3M (25% growth)\nTerminal Value: $29.3M \u00d7 1.02 / (0.12 - 0.02) = $317.39M\nPresent Values:\nPV Year 1-3: $43.95M\nPV Terminal: $173.11M\nEnterprise Value: $217.06M\nRecommendation: Fair value is $217.06M. BUY if trading below $195.65M (10% margin of safety)."
    }
  ]
}

📋 Preview of conversations:

Example 1:
  User: Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3...
  Assistant: DCF Valuation Analysis:
Projected Cash Flows:
Year 1: $18.75M (25% growth)
Year ...

Example 2:
  User: Analyze portfolio with 70% stocks ($70k), 30% bonds ($30k

## 🔄 Step 6: Format Dataset for Training

In [7]:
def format_chat(example):
    messages = example['messages']
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_chat, batched=False)

print("✅ Dataset formatted for training")
print(f"\nFormatted example:")
print(dataset[0]['text'][:500] + "...")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Dataset formatted for training

Formatted example:
<|start_header_id|>user<|end_header_id|>

Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3 years, 12% discount rate, 2% terminal growth<|eot_id|><|start_header_id|>assistant<|end_header_id|>

DCF Valuation Analysis:
Projected Cash Flows:
Year 1: $18.75M (25% growth)
Year 2: $23.44M (25% growth)
Year 3: $29.3M (25% growth)
Terminal Value: $29.3M × 1.02 / (0.12 - 0.02) = $317.39M
Present Values:
PV Year 1-3: $43.95M
PV Terminal: $173.11M
Enterprise Value: $217.06M
R...


## 🏋️ Step 7: Configure Trainer

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
# from peft import LoraConfig # Removed as it's no longer needed here

# No need to reconstruct LoraConfig here as the model is already a PeftModel

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    # peft_config=peft_config, # Removed: Model is now explicitly a PeftModel
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    )
)

print("✅ Trainer configured")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Trainer configured
Effective batch size: 8


## 🚀 Step 8: Start Training!

**This will take 20-30 minutes. Don't close the browser!**

In [9]:
import time

print("🚀 Starting training...\n")
start_time = time.time()

trainer_stats = trainer.train()

elapsed = time.time() - start_time
print(f"\n✅ Training complete!")
print(f"Time: {elapsed/60:.1f} minutes")
print(f"Final loss: {trainer_stats.training_loss:.4f}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Starting training...



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 3 | Total steps = 39
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.794100
2,2.725700
3,2.694300
4,3.055200
5,2.541400
6,2.420000
7,2.229800
8,2.283200
9,2.144200
10,1.667300



✅ Training complete!
Time: 4.2 minutes
Final loss: 1.1446


## 💾 Step 9: Save Model to Google Drive

In [10]:
model.save_pretrained(MODEL_OUTPUT_DIR)
tokenizer.save_pretrained(MODEL_OUTPUT_DIR)

import json
metadata = {
    "model_name": MODEL_NAME,
    "dataset": DATASET_FILENAME,
    "base_model": "unsloth/llama-3-8b-bnb-4bit",
    "training_loss": float(trainer_stats.training_loss),
    "num_examples": len(dataset),
    "num_epochs": NUM_EPOCHS,
    "training_time_minutes": elapsed / 60,
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
}

with open(f"{MODEL_OUTPUT_DIR}/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Model saved to: {MODEL_OUTPUT_DIR}")
print(f"\nMetadata:")
print(json.dumps(metadata, indent=2))

✅ Model saved to: /content/drive/MyDrive/Finetune_Jobs/models/financial gpt

Metadata:
{
  "model_name": "financial gpt",
  "dataset": "dataset-20251125_095111.jsonl",
  "base_model": "unsloth/llama-3-8b-bnb-4bit",
  "training_loss": 1.1446102811739995,
  "num_examples": 100,
  "num_epochs": 3,
  "training_time_minutes": 4.203472924232483,
  "timestamp": "2025-11-25 11:07:00"
}


## 🧪 Step 10: Test the Model

In [11]:
FastLanguageModel.for_inference(model)

test_messages = [
    {"role": "user", "content": "Hello! Can you help me?"}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

print("🤖 Testing model...\n")
print("User: Hello! Can you help me?\n")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_response = response.split("assistant\n")[-1] if "assistant" in response else response

print(f"Assistant: {assistant_response}")
print("\n🎉 All done! Your model is ready to use.")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🤖 Testing model...

User: Hello! Can you help me?

Assistant: user

Hello! Can you help me?assistant: Sure, I'm here to assist.assistant: First, let's create a new database for storing patient information.assistant: Next, we'll define table schemas for patients, doctors, and appointments.assistant: Then, we'll insert sample data into the tables.assistant: Finally, we'll query the database to retrieve patient information.assistant: Here's the SQL code:assistant: -- Database: patient_dataassistant: -- Tables:assistant: patient (id, name, phone)assistant: doctor (id, name, specialty)assistant: appointment (id, date, time, patient_id, doctor_id)assistant: --

🎉 All done! Your model is ready to use.


# Task
To resolve the `ValueError` encountered in the `trainer-cell` and allow `SFTTrainer` to properly recognize the LoRA-adapted model, I will update the notebook in two key steps:

1.  **Modify the `lora-cell`**:
    *   I will add an import for `peft.PeftModel` and `peft.LoraConfig`.
    *   I will define a `LoraConfig` object with the same parameters used for Unsloth's `get_peft_model`.
    *   After the model has been processed by `FastLanguageModel.get_peft_model`, I will explicitly wrap the resulting model in `peft.PeftModel` using the defined `LoraConfig`. This will ensure the model object is unequivocally an instance of `peft.PeftModel`, which `SFTTrainer` expects.
2.  **Revert the `trainer-cell`**:
    *   I will remove the explicit `peft_config` argument from the `SFTTrainer` initialization.
    *   I will also remove the `LoraConfig` import and its definition from this cell, as the model will now be pre-wrapped as a `peft.PeftModel` and `SFTTrainer` will automatically detect its configuration.

After these modifications, I will execute the `lora-cell`, then re-run the `dataset-cell` and `format-cell` (which were successful previously and are prerequisites for the trainer), and finally, run the updated `trainer-cell`.

```python
# lora-cell
from peft import PeftModel, LoraConfig # Add LoraConfig and PeftModel import

# Create the LoraConfig object that defines the PEFT structure
# This config will be used to explicitly wrap the model later.
# Use the same parameters as passed to FastLanguageModel.get_peft_model
lora_config_for_wrapper = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM", # Specify task type for PeftModel
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

# Explicitly wrap the Unsloth-adapted model in a peft.PeftModel.
# This step ensures that 'model' is an instance of peft.PeftModel,
# which SFTTrainer checks for.
model = PeftModel(model, lora_config_for_wrapper)

print("✅ LoRA adapters added and model explicitly wrapped as PeftModel")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# trainer-cell
from trl import SFTTrainer
from transformers import TrainingArguments
# from peft import LoraConfig # Removed as it's no longer needed here

# No need to reconstruct LoraConfig here as the model is already a PeftModel

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    # peft_config=peft_config, # Removed: Model is now explicitly a PeftModel
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

print("✅ Trainer configured")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")
```

## lora-cell-fix

### Subtask:
Modify the `lora-cell` to explicitly wrap the Unsloth-adapted model in a `peft.PeftModel` to make it compatible with `SFTTrainer`'s checks.


**Reasoning**:
To explicitly wrap the Unsloth-adapted model in a `peft.PeftModel` as required by the subtask, I will modify the existing `lora-cell` to include the necessary import, define a `LoraConfig` object, perform the wrapping, and update the confirmation message.



In [12]:
from peft import PeftModel, LoraConfig

lora_config_for_wrapper = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=lora_config_for_wrapper.target_modules,
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

model = PeftModel(model, lora_config_for_wrapper)

print("✅ LoRA adapters added and model explicitly wrapped as PeftModel")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Unsloth: Already have LoRA adapters! We shall skip this step.
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


✅ LoRA adapters added and model explicitly wrapped as PeftModel
Trainable parameters: 41,943,040


**Reasoning**:
The previous step successfully applied the LoRA adapter modifications to the model. Now, as per the overall plan, I will modify the `trainer-cell` by removing the redundant `peft_config` definition and argument, as the model is now explicitly a `PeftModel` instance, which `SFTTrainer` should automatically recognize.



In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
# from peft import LoraConfig # Removed as it's no longer needed here

# No need to reconstruct LoraConfig here as the model is already a PeftModel

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    # peft_config=peft_config, # Removed: Model is now explicitly a PeftModel
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION,
        warmup_steps=WARMUP_STEPS,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    )
)

print("✅ Trainer configured")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION}")

✅ Trainer configured
Effective batch size: 8


## run-dataset-cells

### Subtask:
Run the `dataset-cell` and `format-cell` to ensure the dataset is loaded and formatted correctly, as these steps were previously successful and need to be re-executed before configuring the trainer.


**Reasoning**:
To ensure the dataset is loaded and available for the trainer, I will execute the `dataset-cell` which loads the JSON dataset from the specified path.



In [14]:
from datasets import load_dataset
import json

dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

print(f"✅ Dataset loaded: {len(dataset)} examples")
print(f"\nFirst example:")
print(json.dumps(dataset[0], indent=2))

print(f"\n📋 Preview of conversations:")
for i in range(min(3, len(dataset))):
    messages = dataset[i]['messages']
    user_msg = next((m['content'] for m in messages if m['role'] == 'user'), '')
    assistant_msg = next((m['content'] for m in messages if m['role'] == 'assistant'), '')
    print(f"\nExample {i+1}:")
    print(f"  User: {user_msg[:80]}...")
    print(f"  Assistant: {assistant_msg[:80]}...")

✅ Dataset loaded: 100 examples

First example:
{
  "messages": [
    {
      "role": "user",
      "content": "Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3 years, 12% discount rate, 2% terminal growth"
    },
    {
      "role": "assistant",
      "content": "DCF Valuation Analysis:\nProjected Cash Flows:\nYear 1: $18.75M (25% growth)\nYear 2: $23.44M (25% growth)\nYear 3: $29.3M (25% growth)\nTerminal Value: $29.3M \u00d7 1.02 / (0.12 - 0.02) = $317.39M\nPresent Values:\nPV Year 1-3: $43.95M\nPV Terminal: $173.11M\nEnterprise Value: $217.06M\nRecommendation: Fair value is $217.06M. BUY if trading below $195.65M (10% margin of safety)."
    }
  ]
}

📋 Preview of conversations:

Example 1:
  User: Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3...
  Assistant: DCF Valuation Analysis:
Projected Cash Flows:
Year 1: $18.75M (25% growth)
Year ...

Example 2:
  User: Analyze portfolio with 70% stocks ($70k), 30% bonds ($30k

**Reasoning**:
To format the loaded dataset for training, I will execute the `format-cell` which applies the chat template to each example.



In [15]:
def format_chat(example):
    messages = example['messages']
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_chat, batched=False)

print("✅ Dataset formatted for training")
print(f"\nFormatted example:")
print(dataset[0]['text'][:500] + "...")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

✅ Dataset formatted for training

Formatted example:
<|start_header_id|>user<|end_header_id|>

Calculate DCF valuation for a company with $15M revenue, 25% annual growth for 3 years, 12% discount rate, 2% terminal growth<|eot_id|><|start_header_id|>assistant<|end_header_id|>

DCF Valuation Analysis:
Projected Cash Flows:
Year 1: $18.75M (25% growth)
Year 2: $23.44M (25% growth)
Year 3: $29.3M (25% growth)
Terminal Value: $29.3M × 1.02 / (0.12 - 0.02) = $317.39M
Present Values:
PV Year 1-3: $43.95M
PV Terminal: $173.11M
Enterprise Value: $217.06M
R...


## run-trainer-cell

### Subtask:
Run the `trainer-cell` with the updated code to configure the trainer.


## Summary:

### Data Analysis Key Findings
*   The `lora-cell` was successfully updated to explicitly wrap the model using `peft.PeftModel` with a specified `LoraConfig`, ensuring the model is correctly recognized by `SFTTrainer`.
*   A `UserWarning` was observed during the explicit wrapping in the `lora-cell`, indicating that the model already contained a `peft_config` attribute before the explicit `PeftModel` wrapping, which might lead to multiple adapters.
*   After the `lora-cell` modifications, the model reported 41,943,040 trainable parameters.
*   The `trainer-cell` was successfully reconfigured by removing the redundant `peft_config` argument and `LoraConfig` import, as the model was already pre-wrapped as a `peft.PeftModel`.
*   The `SFTTrainer` was initialized without errors, confirming "✅ Trainer configured" and an effective batch size of 8.
*   The `dataset-cell` successfully loaded 10 examples from the specified JSON file.
*   The `format-cell` successfully processed the dataset, applying the chat template and creating a formatted "text" field for training.

### Insights or Next Steps
*   Investigate the `UserWarning` regarding multiple `peft_config` attributes to ensure optimal PEFT model configuration and avoid potential conflicts or inefficiencies during training.
*   Proceed with training the model using the successfully configured `SFTTrainer` and the prepared dataset.
